In [ ]:
# Install necessary libraries
!pip install earthengine-api geemap folium --quiet

import ee
import geemap
import folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00


In [ ]:
# Authenticate with Google Earth Engine
ee.Authenticate()
# Initialize Earth Engine with your project ID
ee.Initialize(project='ee-kgyamfi')

In [ ]:
# Define Polygons and Load Hansen GFC Image

# Load asset and GFC image
polygons = ee.FeatureCollection("projects/ee-kgyamfi/assets/over4hpoly")
gfc_image = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for UMD/hansen/global_forest_change_2023_v1_11! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2023_v1_11

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
 # Forest Loss Processing from 2021–2023

start_year = 2021
end_year = 2023

combined_loss_image = ee.Image(0)  # Initialize empty image

# Loop through years
for year in range(start_year, end_year + 1):
    loss_year_image = gfc_image.select('lossyear').eq(year - 2000)  # GFC 'lossyear' is 1-based from 2001

    # Convert pixel loss to area in hectares
    loss_area_sqm = loss_year_image.multiply(ee.Image.pixelArea())
    loss_area_ha = loss_area_sqm.divide(10000).clip(polygons)

    combined_loss_image = combined_loss_image.add(loss_area_ha)

    # Visualize yearly forest loss (in red)
    vis_params = {"min": 0, "max": 100, "palette": ["FF0000"]}
    layer_name = f"Forest Loss {year}"
   # Map.addLayer(loss_area_ha, vis_params, layer_name)

In [ ]:
# Compute Forest Loss by Polygon (Exportable Table)

# Use 'loss' image + lossyear band for per-feature reduction
loss_image = gfc_image.select('loss')
loss_area = loss_image.multiply(ee.Image.pixelArea()).divide(10000).clip(polygons)

loss_year_image = gfc_image.select('lossyear')

loss_by_year = loss_area.addBands(loss_year_image)

# Reduce per polygon
loss_per_polygon = loss_by_year.reduceRegions(
    collection=polygons,
    reducer=ee.Reducer.sum(),
    scale=30
)

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=loss_per_polygon,
    description='forest_loss_per_polygon_21',
    folder='GEE_exports',
    fileFormat='CSV'
)
task.start()


In [ ]:
Map = geemap.Map(center=[0, 0], zoom=6)
Map.centerObject(polygons, 8)
Map.addLayer(polygons, {}, "Forest Polygons")
Map.addLayer(combined_loss_image, {"min": 0, "max": 300, "palette": ["white", "red"]}, "Combined Forest Loss (2021–2023)")
Map


Map(center=[6.8648228800036435, -2.6979612175676193], controls=(WidgetControl(options=['position', 'transparen…